i = Image(r"D:\OneDrive - stu.xidian.edu.cn\图片\20190515_180216304_iOS.jpg")  
i.read_exif()  
i.modify_exif({'Exif.Photo.DateTimeOriginal': '2019:05:15 18:00:17'})  
i.read_exif()  

{'Exif.Image.DateTime': '2019:05:15 18:02:16',  
 'Exif.Image.ExifTag': '58',  
 'Exif.Photo.DateTimeOriginal': '2019:05:15 18:02:16',  
 'Exif.Thumbnail.JPEGInterchangeFormat': '126',  
 'Exif.Thumbnail.JPEGInterchangeFormatLength': '0'}

In [7]:
# coding = utf-8
from pyexiv2 import Image
import os
import re
import time
import os


# 时间范式
time1 = re.compile(r"15\d{11}")  # 注意后三位是多余的
time2 = re.compile(r"20\d{6}_\d{6}")
time3 = re.compile(r"IMG_20\d{6}_\d{6}")
time4 = re.compile(r"mmexport20\d{12}")
time5 = re.compile(r"(S|s)creenshot_20\d{6}-\d{6}")
time6 = re.compile(r"wx_camera_15\d{11}")  # 注意后三位是多余的
TIME_PATTERNS = [time1, time2, time3, time4, time5, time6]

In [14]:
class my_extractor(Image):
    def __init__(self, file_name):
        self.file_name = file_name
        self.image = Image(file_name)
        self.exif = self.image.read_exif()

    # 2个从文件名提取原始时间的函数
    def extract_time_from_datetime_of(file_name):
        """从file_name中直接提取时间信息，转化成时间字符串形式返回"""
        time_example = r"(20\d\d)(\d\d)(\d\d)(?:_|-|)(\d\d)(\d\d)(\d\d)"
        time_raw = re.findall(time_example, file_name)[0]
        date_time = time_raw[0] + ":"
        for i in range(1, 6):
            date_time += time_raw[i]
            if i == 2:
                date_time += " "
            elif i == 5:
                pass
            else:
                date_time += ":"
        return date_time

    def extract_time_from_timestrip_of(file_name):
        """
        从file_name中提取出时间戳信息,然后转化成时间字符串形式返回
        
        """
        time_example = r"15\d{11}"
        time_splited = int(re.findall(time_example, file_name)[0][:-3])
        if time_splited <= now_time:
            # 转化为'2019:05:15 18:00:17'格式
            date_time = time.strftime("%Y:%m:%d %H:%M:%S", time.localtime(time_splited))
            return date_time
        else:
            print("Wrong input you dumbass!")
            return None

    def get_time_from_FileName_Of(self, TIME_PATTERNS):
        """
        根据图片的名字获取图片本来的exif时间
        time_Patterns:list, 元素为时间模式，来自于正则表达式
        image_file:str  为图片名字，例如"20190515_180216304_iOS.jpg"
        可以用来修改的名字有
        "1513859190222.jpeg"
        "20150829_170328000.png"
        "IMG_20190520_120207.jpg"
        "mmexport20171117093617.jpg"
        "Screenshot_20190313-214413_WeChat.jpg"
        "wx_camera_1527957631631.jpg"
        """
        now_time = time.time()
        # 开始提取操作
        isThisFileHasBeenExtracted = False  # 标记当前文件未被提取
        # 拿6种模式来匹配文件名
        for time_rePattern in TIME_PATTERNS:
            if not isThisFileHasBeenExtracted:
                # 当这个文件还没有被提取的时间的时候再往下走
                if re.match(time_rePattern, self.file_name):
                    self.date_time = (
                        extract_time_from_timestrip_of(self.file_name)
                        if (time_rePattern == time1 or time_rePattern == time6)
                        else extract_time_from_datetime_of(self.file_name)
                    )
                    isThisFileHasBeenExtracted = True
                    break  # 匹配到了就直接跳出循环就好
                else:  # 如果没有匹配，那么就进行下一次循环
                    continue
        if isThisFileHasBeenExtracted:
            # 如果匹配到了那么直接返回
            return self.date_time
        # print("Time of {} is {}".format(file_name, date_time))
        else:
            # 没有匹配到那就开骂吧
            print("Nothing Happened to {}. You such a DUMBASS.".format(self.file_name))
            return None

    def need_modify(self):
        """
        判断这个图片的EXIF信息是否需要修改，如果不需要的话就直接滚犊子吧,如果需要在进行modify_exit的操作.
        要传入Image.read_exif()这个词典，然后对比时间
        """
        # 首先看看是否存在创建时间，如果不存在直接就要搞
        if "Exif.Photo.DateTimeOriginal" in self.exif.keys():
            # 存在创建时间的话看看和我们需要对比的时间是否一致，不一致也得搞
            if self.exif[
                "Exif.Photo.DateTimeOriginal"
            ] == self.get_time_from_FileName_Of(TIME_PATTERNS):
                # 相等的话说明不用改,返回个false
                return False
        else:
            # 但凡有一点不让人满意那么就需要修改
            return True

    def change_exif(self):
        """
        image_file为图片名字，例如"20190515_180216304_iOS.jpg"
        注意，应该拍摄日期和创建日期，这2个都要修改的！！！！！！
        如果本身有这些信息的话，那么就不需要修改
        time应为需要修改成的时间，严格按照如下格式'2019:05:15 18:00:17'。如果不是就直接报错。
        没有任何返回值，直接修改。"""
        if self.need_modify():
            self.modify_exif({"Exif.Photo.DateTimeOriginal": self.date_time})
            return print(
                "Time of {} has been changed to {}".format(
                    self.file_name, self.date_time
                )
            )
        else:
            return print("Error! Can't modify time of {}".format(self.file_name))

TypeError: 'str' object is not callable

In [12]:
print(os.getcwd())
os.chdir(r"D:\OneDrive - stu.xidian.edu.cn\图片")
print(os.getcwd())

d:\OneDrive\ModifyEXIF
D:\OneDrive - stu.xidian.edu.cn\图片


In [15]:
test_file = "20190527_180319.jpg"
test = my_extractor(test_file)
test.change_exif()

AttributeError: 'my_extractor' object has no attribute 'date_time'

In [19]:
test.get_time_from_FileName_Of(TIME_PATTERNS)

TypeError: first argument must be string or compiled pattern

In [ ]:


path = "D:/OneDrive - stu.xidian.edu.cn/图片" #文件夹目录
root_files= os.listdir(path)
for name in root_files:#遍历文件夹
    #如果是文件，那就直接修改
    

In [ ]:
path = "D:/Python34/news"  # 文件夹目录
files = os.listdir(path)  # 得到文件夹下的所有文件名称
s = []
for file in files:  # 遍历文件夹
    if not os.path.isdir(file):  # 判断是否是文件夹，不是文件夹才打开
        f = open(path + "/" + file)
        # 打开文件
        iter_f = iter(f)
        # 创建迭代器
        str = ""
        for line in iter_f:  # 遍历文件，一行行遍历，读取文本
            str = str + line
        s.append(str)  # 每个文件的文本存到list中
print(s)  # 打印结果

In [ ]:
file = 